In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [6]:
subject = 'Mato Grosso Do Sul - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [7]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_MS.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Educacao,Mato Grosso Do Sul - Produção de Cimento (t),Mato Grosso Do Sul - Consumo de Cimento (t)
0,2014,0.751382,5.543864e+07,2.276794e+06,16.985287,4.261295e+07,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,33.031265,953.098166,1382.443280,23.914606,16.608338,67.060540,3.088252,0.704405,886.944606,995.222143
1,2015,0.742455,5.607449e+07,2.266815e+06,16.994005,4.270203e+07,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,31.224126,953.111147,1377.219450,23.862880,16.783027,67.636795,3.072316,0.713916,822.691374,951.041314
2,2016,0.743951,5.656732e+07,2.251674e+06,17.013733,4.279700e+07,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,29.308459,952.984733,1353.708395,23.814967,16.659956,67.363296,3.049349,0.718181,737.430411,904.537543
3,2017,0.744219,5.674497e+07,2.236955e+06,17.015525,4.281848e+07,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,27.027249,953.036051,1260.888062,23.920271,16.556341,66.782526,2.972744,0.720015,688.444085,870.341000
4,2018,0.744784,5.672054e+07,2.234064e+06,17.010396,4.279266e+07,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,27.485116,953.178485,1250.680996,24.117233,16.399497,65.653473,2.903867,0.721107,702.799997,869.680000
5,2019,0.744310,5.624466e+07,2.233988e+06,16.948562,4.260557e+07,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,29.097428,953.114806,1333.291299,24.276760,15.888657,63.576346,2.603781,0.721663,762.792596,902.144000
6,2020,0.743611,5.565791e+07,2.232222e+06,16.918732,4.247150e+07,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,30.513520,953.063961,1377.234292,24.380377,15.817375,62.811800,2.108572,0.720814,834.473939,1038.045000
7,2021,0.743576,5.432535e+07,2.230057e+06,16.833993,4.211105e+07,6.638811e+08,4.421848e+09,1.933031e+10,4.488070e+10,...,34.097499,953.096180,1403.018305,24.285568,15.783928,62.584820,2.048248,0.718433,865.320803,1092.056000
8,2022,0.744537,5.272243e+07,2.227392e+06,16.730205,4.167110e+07,7.010332e+08,4.533216e+09,1.997888e+10,4.699319e+10,...,36.087097,953.164445,1411.847677,24.084737,15.520435,62.203762,2.265775,0.716382,869.491750,1152.060000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Mato Grosso do Sul - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Educacao,Mato Grosso Do Sul - Produção de Cimento (t)
0,0.751382,5.543864e+07,2.276794e+06,16.985287,4.261295e+07,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,4.570163e+10,...,0.804248,33.031265,953.098166,1382.443280,23.914606,16.608338,67.060540,3.088252,0.704405,886.944606
1,0.742455,5.607449e+07,2.266815e+06,16.994005,4.270203e+07,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,3.539769e+10,...,0.812518,31.224126,953.111147,1377.219450,23.862880,16.783027,67.636795,3.072316,0.713916,822.691374
2,0.743951,5.656732e+07,2.251674e+06,17.013733,4.279700e+07,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,3.407411e+10,...,0.810293,29.308459,952.984733,1353.708395,23.814967,16.659956,67.363296,3.049349,0.718181,737.430411
3,0.744219,5.674497e+07,2.236955e+06,17.015525,4.281848e+07,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,3.680756e+10,...,0.807410,27.027249,953.036051,1260.888062,23.920271,16.556341,66.782526,2.972744,0.720015,688.444085
4,0.744784,5.672054e+07,2.234064e+06,17.010396,4.279266e+07,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,4.727230e+10,...,0.803902,27.485116,953.178485,1250.680996,24.117233,16.399497,65.653473,2.903867,0.721107,702.799997
5,0.744310,5.624466e+07,2.233988e+06,16.948562,4.260557e+07,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,7.227332e+10,...,0.799711,29.097428,953.114806,1333.291299,24.276760,15.888657,63.576346,2.603781,0.721663,762.792596
6,0.743611,5.565791e+07,2.232222e+06,16.918732,4.247150e+07,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,9.809293e+10,...,0.794776,30.513520,953.063961,1377.234292,24.380377,15.817375,62.811800,2.108572,0.720814,834.473939


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     951.041314
1     904.537543
2     870.341000
3     869.680000
4     902.144000
5    1038.045000
6    1092.056000
7    1152.060000
8            NaN
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Mato Grosso do Sul - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Educacao,Mato Grosso Do Sul - Produção de Cimento (t)
0,2.163760,-1.889659,1.596457,-0.398871,-1.242187,1.779974,0.019134,2.003355,-1.880579,0.034168,...,-0.492000,1.688630,0.176979,1.067885,-0.433760,0.434302,0.515307,0.839767,-2.023704,1.737248
1,-0.952476,-0.492184,1.000830,-0.024883,-0.222364,0.870373,-0.621387,0.401153,0.111653,-0.753105,...,1.446553,0.817326,0.387075,0.968396,-0.754988,1.037856,0.930593,0.744089,-0.438690,0.807779
2,-0.430335,0.590955,0.097064,0.821493,0.865048,-0.328802,-0.753466,-0.407217,1.154682,-0.854233,...,0.924859,-0.106304,-1.658944,0.520626,-1.052530,0.612643,0.733492,0.606194,0.272127,-0.425583
3,-0.336706,0.981385,-0.781533,0.898363,1.110908,-0.896969,-0.809829,-0.830898,0.955301,-0.645384,...,0.249191,-1.206179,-0.828361,-1.247146,-0.398580,0.254651,0.314952,0.146254,0.577883,-1.134206
4,-0.139369,0.927701,-0.954156,0.678304,0.815356,-0.983185,0.071542,-0.963455,0.100493,0.154175,...,-0.573139,-0.985421,1.476955,-1.441541,0.824583,-0.287246,-0.498716,-0.267290,0.759925,-0.926537
5,-0.304875,-0.118198,-0.958663,-1.974406,-1.326762,-0.441390,2.094006,-0.202939,-0.441549,2.064379,...,-1.555465,-0.208052,0.446296,0.131780,1.815275,-2.052206,-1.995628,-2.069014,0.852459,-0.058701


In [11]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0     951.041314
1     904.537543
2     870.341000
3     869.680000
4     902.144000
5    1038.045000
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [12]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Mato Grosso do Sul - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Educacao,Mato Grosso Do Sul - Produção de Cimento (t)
6,-0.497579,-1.150596,-0.913982,-1.900575,-1.798599,1.16956,2.035192,0.985175,-0.705937,2.048732,...,-1.753438,0.432589,-0.345205,0.842152,1.66748,-1.60644,-1.69867,-2.169028,0.635696,0.849456


In [13]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    1092.056
Name: Mato Grosso Do Sul - Consumo de Cimento (t), dtype: float64

In [14]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=500, verbose=1)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4096, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(2048, activation='tanh'),
        tf.keras.layers.Dense(512, activation='tanh'),
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [15]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[90110644, 524641069, 2421908983, 54920652, 1232197049, 1971315224, 1737549255, 1788884090, 4060337140, 2454631780]


Step: 0 ___________________________________________


2023-08-19 19:59:51.522343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-19 19:59:51.522699: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 2141: early stopping
val_loss: 4617.88427734375
winner_seed: 90110644


Step: 1 ___________________________________________
Epoch 3063: early stopping
val_loss: 4617.892578125


Step: 2 ___________________________________________
Epoch 2126: early stopping
val_loss: 4617.88427734375


Step: 3 ___________________________________________
Epoch 3914: early stopping
val_loss: 4617.88427734375


Step: 4 ___________________________________________
Epoch 2005: early stopping
val_loss: 4617.88427734375


Step: 5 ___________________________________________
Epoch 1745: early stopping
val_loss: 2824.944580078125
winner_seed: 1971315224


Step: 6 ___________________________________________
Epoch 2090: early stopping
val_loss: 4617.892578125


Step: 7 ___________________________________________
Epoch 1155: early stopping
val_loss: 5403.021484375


Step: 8 ___________________________________________
Epoch 1759: early stopping
val_loss: 10287.0283203125


Step: 9 _______________________________

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 1s 539ms/step - loss: 854654.9375 - val_loss: 1043737.8750
Epoch 2/5000
1/1 [==============================] - 0s 65ms/step - loss: 824055.6875 - val_loss: 426743.6875
Epoch 3/5000
1/1 [==============================] - 0s 65ms/step - loss: 587819.0625 - val_loss: 482743.8750
Epoch 4/5000
1/1 [==============================] - 0s 68ms/step - loss: 873842.7500 - val_loss: 709111.1875
Epoch 5/5000
1/1 [==============================] - 0s 64ms/step - loss: 804178.0000 - val_loss: 804097.8125
Epoch 6/5000
1/1 [==============================] - 0s 66ms/step - loss: 780015.0000 - val_loss: 829690.0625
Epoch 7/5000
1/1 [==============================] - 0s 71ms/step - loss: 760150.1875 - val_loss: 830853.9375
Epoch 8/5000
1/1 [==============================] - 0s 80ms/step - loss: 744217.2500 - val_loss: 823246.3125
Epoch 9/5000
1/1 [==============================] - 0s 69ms/step - loss: 729009.0000 - val_loss: 811203.6875
Epoch 10/5000
1/1

1/1 [==============================] - 0s 59ms/step - loss: 193875.7344 - val_loss: 238278.7031
Epoch 76/5000
1/1 [==============================] - 0s 59ms/step - loss: 190033.8594 - val_loss: 234050.9062
Epoch 77/5000
1/1 [==============================] - 0s 66ms/step - loss: 186268.4375 - val_loss: 229902.6875
Epoch 78/5000
1/1 [==============================] - 0s 64ms/step - loss: 182577.9531 - val_loss: 225832.4531
Epoch 79/5000
1/1 [==============================] - 0s 59ms/step - loss: 178960.9219 - val_loss: 221838.7500
Epoch 80/5000
1/1 [==============================] - 0s 59ms/step - loss: 175415.8594 - val_loss: 217920.0625
Epoch 81/5000
1/1 [==============================] - 0s 61ms/step - loss: 171941.3594 - val_loss: 214075.0312
Epoch 82/5000
1/1 [==============================] - 0s 59ms/step - loss: 168535.9531 - val_loss: 210302.0625
Epoch 83/5000
1/1 [==============================] - 0s 59ms/step - loss: 165198.3594 - val_loss: 206599.9844
Epoch 84/5000
1/1 [=====

Epoch 150/5000
1/1 [==============================] - 0s 59ms/step - loss: 43570.1992 - val_loss: 66485.7266
Epoch 151/5000
1/1 [==============================] - 0s 63ms/step - loss: 42719.4062 - val_loss: 65434.2891
Epoch 152/5000
1/1 [==============================] - 0s 60ms/step - loss: 41885.5508 - val_loss: 64401.5781
Epoch 153/5000
1/1 [==============================] - 0s 61ms/step - loss: 41068.2812 - val_loss: 63387.3438
Epoch 154/5000
1/1 [==============================] - 0s 61ms/step - loss: 40267.2734 - val_loss: 62391.1602
Epoch 155/5000
1/1 [==============================] - 0s 58ms/step - loss: 39482.2109 - val_loss: 61412.6680
Epoch 156/5000
1/1 [==============================] - 0s 59ms/step - loss: 38712.7656 - val_loss: 60451.6328
Epoch 157/5000
1/1 [==============================] - 0s 67ms/step - loss: 37958.6250 - val_loss: 59507.6484
Epoch 158/5000
1/1 [==============================] - 0s 65ms/step - loss: 37219.4961 - val_loss: 58580.3945
Epoch 159/5000
1/1 

1/1 [==============================] - 0s 59ms/step - loss: 10284.6836 - val_loss: 22343.2793
Epoch 226/5000
1/1 [==============================] - 0s 66ms/step - loss: 10096.2744 - val_loss: 22056.2207
Epoch 227/5000
1/1 [==============================] - 0s 68ms/step - loss: 9911.6104 - val_loss: 21773.8633
Epoch 228/5000
1/1 [==============================] - 0s 64ms/step - loss: 9730.6240 - val_loss: 21496.1172
Epoch 229/5000
1/1 [==============================] - 0s 67ms/step - loss: 9553.2354 - val_loss: 21222.9102
Epoch 230/5000
1/1 [==============================] - 0s 67ms/step - loss: 9379.3818 - val_loss: 20954.1523
Epoch 231/5000
1/1 [==============================] - 0s 69ms/step - loss: 9208.9844 - val_loss: 20689.7773
Epoch 232/5000
1/1 [==============================] - 0s 66ms/step - loss: 9041.9834 - val_loss: 20429.6973
Epoch 233/5000
1/1 [==============================] - 0s 61ms/step - loss: 8878.3047 - val_loss: 20173.8477
Epoch 234/5000
1/1 [=====================

1/1 [==============================] - 0s 58ms/step - loss: 2871.8284 - val_loss: 9608.9297
Epoch 302/5000
1/1 [==============================] - 0s 59ms/step - loss: 2830.9358 - val_loss: 9521.1455
Epoch 303/5000
1/1 [==============================] - 0s 60ms/step - loss: 2790.8555 - val_loss: 9434.6406
Epoch 304/5000
1/1 [==============================] - 0s 89ms/step - loss: 2751.5754 - val_loss: 9349.3936
Epoch 305/5000
1/1 [==============================] - 0s 66ms/step - loss: 2713.0781 - val_loss: 9265.3789
Epoch 306/5000
1/1 [==============================] - 0s 65ms/step - loss: 2675.3467 - val_loss: 9182.5732
Epoch 307/5000
1/1 [==============================] - 0s 59ms/step - loss: 2638.3640 - val_loss: 9100.9668
Epoch 308/5000
1/1 [==============================] - 0s 57ms/step - loss: 2602.1194 - val_loss: 9020.5244
Epoch 309/5000
1/1 [==============================] - 0s 59ms/step - loss: 2566.5911 - val_loss: 8941.2471
Epoch 310/5000
1/1 [==============================] 

...

1/1 [==============================] - 0s 59ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1375/5000
1/1 [==============================] - 0s 60ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1376/5000
1/1 [==============================] - 0s 60ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1377/5000
1/1 [==============================] - 0s 58ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1378/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1379/5000
1/1 [==============================] - 0s 60ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1380/5000
1/1 [==============================] - 0s 62ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1381/5000
1/1 [==============================] - 0s 61ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1382/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1383/5000
1/1 [==============================] 

1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9509
Epoch 1451/5000
1/1 [==============================] - 0s 67ms/step - loss: 816.9064 - val_loss: 2824.9509
Epoch 1452/5000
1/1 [==============================] - 0s 64ms/step - loss: 816.9063 - val_loss: 2824.9509
Epoch 1453/5000
1/1 [==============================] - 0s 65ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1454/5000
1/1 [==============================] - 0s 64ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1455/5000
1/1 [==============================] - 0s 90ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1456/5000
1/1 [==============================] - 0s 66ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1457/5000
1/1 [==============================] - 0s 65ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1458/5000
1/1 [==============================] - 0s 62ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1459/5000
1/1 [==============================] 

1/1 [==============================] - 0s 67ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1527/5000
1/1 [==============================] - 0s 64ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1528/5000
1/1 [==============================] - 0s 61ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1529/5000
1/1 [==============================] - 0s 62ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1530/5000
1/1 [==============================] - 0s 61ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1531/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1532/5000
1/1 [==============================] - 0s 58ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1533/5000
1/1 [==============================] - 0s 58ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1534/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1535/5000
1/1 [==============================] 

1/1 [==============================] - 0s 61ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1603/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1604/5000
1/1 [==============================] - 0s 65ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1605/5000
1/1 [==============================] - 0s 58ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1606/5000
1/1 [==============================] - 0s 57ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1607/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1608/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1609/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1610/5000
1/1 [==============================] - 0s 63ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1611/5000
1/1 [==============================] 

1/1 [==============================] - 0s 63ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1679/5000
1/1 [==============================] - 0s 63ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1680/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1681/5000
1/1 [==============================] - 0s 64ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1682/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9063 - val_loss: 2824.9446
Epoch 1683/5000
1/1 [==============================] - 0s 60ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1684/5000
1/1 [==============================] - 0s 58ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1685/5000
1/1 [==============================] - 0s 60ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1686/5000
1/1 [==============================] - 0s 59ms/step - loss: 816.9064 - val_loss: 2824.9446
Epoch 1687/5000
1/1 [==============================] 

In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 78ms/step
[1092.056](test_target) - [[984.89484]](prediction) = 107.16116357421879


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Mato Grosso do Sul - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Educacao,Mato Grosso Do Sul - Produção de Cimento (t)
0,1.396804,-1.273074,1.132900,-1.041037,-1.211443,1.165029,1.396038,1.335266,-1.330895,1.406550,...,-1.365564,1.212672,0.589712,0.905726,1.240069,-1.029443,-1.244937,1.144743,-1.348006,1.163561
1,-0.889984,0.103188,0.166629,-0.308446,-0.026185,0.111769,-0.502296,-0.264151,0.251276,-0.575945,...,1.001256,0.023800,0.818328,0.487746,-0.031246,1.354475,1.203501,0.146776,0.303651,0.114348
2,-0.506821,1.169886,-1.299528,1.349483,1.237627,-1.276798,-0.893743,-1.071116,1.079619,-0.830605,...,0.364309,-1.236471,-1.408041,-1.393472,-1.208823,-0.325032,0.041437,-1.291519,1.044355,-1.277909


train_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
0,951.041314
1,904.537543
2,870.341000


test_input:


...

,Mato Grosso do Sul - IDH Renda,Mato Grosso do Sul - PIB - Estadual,Mato Grosso do Sul - PIB - Construção Civil,Mato Grosso do Sul - PIB - Per Capita,Mato Grosso do Sul - PIB - Preços de Mercado,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),Contratação SFH (Bi R$),...,Mato Grosso do Sul - IDH Longevidade,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Mato Grosso do Sul - IDH Educacao,Mato Grosso Do Sul - Produção de Cimento (t)
6,-0.497579,-1.150596,-0.913982,-1.900575,-1.798599,1.16956,2.035192,0.985175,-0.705937,2.048732,...,-1.753438,0.432589,-0.345205,0.842152,1.66748,-1.60644,-1.69867,-2.169028,0.635696,0.849456


test_target:


,Mato Grosso Do Sul - Consumo de Cimento (t)
6,1092.056


Epoch 1745: early stopping
1/1 [==============================] - 0s 43ms/step
Error: 107.16116357421879




[887.2401733398438, 870.0075073242188, 906.2072143554688, 984.8948364257812]

In [21]:
display(targets)
display(predictions)

[869.68, 902.144, 1038.045, 1092.056]

[887.2401733398438, 870.0075073242188, 906.2072143554688, 984.8948364257812]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

72.17390380859379

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.07398799701028984